# Chess Neural Network

Building a neural network that learns to play chess based on my gameplay data.

## Goal
Train a move prediction model to create an AI that plays like me.




In [18]:
# Importing necessary libraries
import pandas as pd
import ast
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader


## Step 2: Load Chess Data

Load cleaned data from csv file


In [19]:
df = pd.read_csv("/Users/riteshbhandari/Documents/Dokumentit – Ritesh - MacBook Pro/GitHub/Chess-engine/src/data-analysis/cleaned_data.csv")
df.head()

,game_id,moves,num_moves,first_move
0,123118274906,"[('white', 'e4', True), ('black', 'e6', False)...",90,e4
1,123118510404,"[('white', 'd4', False), ('black', 'c5', True)...",141,d4
2,123118790014,"[('white', 'e4', False), ('black', 'e5', True)...",46,e4
3,123158939328,"[('white', 'e4', False), ('black', 'd5', True)...",88,e4
4,123160166430,"[('white', 'e4', True), ('black', 'e5', False)...",110,e4


# Step 2.1:  Further Pre-processing


In [20]:
# convert string representation of list to actual list
df["moves"] = df["moves"].apply(ast.literal_eval)

# saving all the moves to single list to be encoded 
every_move = []
for game in df["moves"]:
    for move in game:
        every_move.append(move[1])

every_move[:10]

['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']

In [21]:
# getting all the unique moves
unique_moves = set(every_move)  # just the unique moves
print("Number of different moves:", len(unique_moves))
print()

# give move a number
move_to_number = {}

# turning integer back to moves (for future use)
number_to_move= {}

for i, move in enumerate(unique_moves):
    move_to_number[move] = i
    number_to_move[i] = move

# turning all the numbers into integers
number_moves = []
for move in every_move:
    number_moves.append(move_to_number[move])

# first 10 moves
print("First 10 moves as numbers: ")
print(number_moves[:10])
print()

# first 10 original moves
print("First 10 original moves: ")
print(every_move[:10]) 

Number of different moves: 1927

First 10 moves as numbers: 
[724, 80, 818, 65, 1455, 1059, 504, 997, 1541, 283]

First 10 original moves: 
['e4', 'e6', 'd4', 'Qh4', 'Nc3', 'f5', 'Nf3', 'Qe7', 'e5', 'Qb4']


In [ ]:
# Create colors per game
colors_per_game = []

for game in df["moves"]:
    
    game_colors = []
    for move in game:
        if move[0] == "white":
            game_colors.append(1)
        else:  # black
            game_colors.append(0)
    colors_per_game.append(game_colors)

# Add to DataFrame
df["colors"] = colors_per_game

# Check first row
print(df[["moves", "colors"]].iloc[0])


moves     [(white, e4, True), (black, e6, False), (white...
colors    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [ ]:
# turn white or black to integers ( 1  = (White), 0 = (Black) )
teoriat_moves_per_game = []

for game in df["moves"]:
    
    game_teoriat = []
    for move in game:
        if move[2] == True:
            game_teoriat.append(1)
        else:
            game_teoriat.append(0)
    teoriat_moves_per_game.append(game_teoriat)

# Add to DataFrame
df["teoriat_moves"] = teoriat_moves_per_game

# Check first row
print(df[["moves", "teoriat_moves"]].iloc[0])

moves            [(white, e4, True), (black, e6, False), (white...
teoriat_moves    [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, ...
Name: 0, dtype: object


In [ ]:
#combine all the features into a single list of tuples seperated by games 
combined_features_per_game = []

for game_idx in range(len(df)):
    
    game_combined = []
    moves = df["moves"].iloc[game_idx]
    colors = df["colors"].iloc[game_idx]
    teoriat = df["teoriat_moves"].iloc[game_idx]
    
    for i in range(len(moves)):
        # (color, move_as_integer, your_move)
        game_combined.append((
            colors[i],
            move_to_number[moves[i][1]],  # convert move to integer
            teoriat[i]
        ))
    # Append the combined features for the game
    combined_features_per_game.append(game_combined)

# Add to DataFrame
df["game_data"] = combined_features_per_game

# Check first game data
print(combined_features_per_game[0])

[(1, 724, 1), (0, 80, 0), (1, 818, 1), (0, 65, 0), (1, 1455, 1), (0, 1059, 0), (1, 504, 1), (0, 997, 0), (1, 1541, 1), (0, 283, 0), (1, 1378, 1), (0, 915, 0), (1, 1443, 1), (0, 1139, 0), (1, 970, 1), (0, 1710, 0), (1, 1775, 1), (0, 237, 0), (1, 601, 1), (0, 131, 0), (1, 303, 1), (0, 102, 0), (1, 163, 1), (0, 1468, 0), (1, 563, 1), (0, 797, 0), (1, 1918, 1), (0, 1302, 0), (1, 654, 1), (0, 447, 0), (1, 303, 1), (0, 1329, 0), (1, 598, 1), (0, 1031, 0), (1, 1008, 1), (0, 1411, 0), (1, 1271, 1), (0, 280, 0), (1, 744, 1), (0, 1531, 0), (1, 1867, 1), (0, 1319, 0), (1, 311, 1), (0, 483, 0), (1, 1008, 1), (0, 291, 0), (1, 1214, 1), (0, 1848, 0), (1, 126, 1), (0, 1126, 0), (1, 1880, 1), (0, 1657, 0), (1, 396, 1), (0, 237, 0), (1, 1876, 1), (0, 1637, 0), (1, 1008, 1), (0, 1785, 0), (1, 744, 1), (0, 1023, 0), (1, 1539, 1), (0, 1485, 0), (1, 995, 1), (0, 829, 0), (1, 1581, 1), (0, 769, 0), (1, 1375, 1), (0, 1772, 0), (1, 1539, 1), (0, 769, 0), (1, 1375, 1), (0, 1772, 0), (1, 1412, 1), (0, 817, 0), 

In [25]:
# lets drop the columns with unneccessary data 
df = df.drop(columns=["moves", "colors", "teoriat_moves", "first_move","num_moves"])
df.head()

,game_id,game_data
0,123118274906,"[(1, 724, 1), (0, 80, 0), (1, 818, 1), (0, 65,..."
1,123118510404,"[(1, 818, 0), (0, 70, 1), (1, 168, 0), (0, 154..."
2,123118790014,"[(1, 724, 0), (0, 1541, 1), (1, 504, 0), (0, 4..."
3,123158939328,"[(1, 724, 0), (0, 168, 1), (1, 1541, 0), (0, 9..."
4,123160166430,"[(1, 724, 1), (0, 1541, 0), (1, 868, 1), (0, 3..."


## Step 2.2: Data Pipeline

### 2.2-1 Dataset Class
Organize chess games into training sequences with sliding windows and padding.

### 2.2-2  Purge K-Fold Cross-Validation
Setup cross-validation splits for robust model evaluation.

In [ ]:
unique_moves = set()
for game in df["game_data"]:
    for (color, move, your_move) in game:
        unique_moves.add(move)

print(f"Number of unique moves: {len(unique_moves)}")
print(f"Move range: {min(unique_moves)} to {max(unique_moves)}")

Number of unique moves: 1927
Move range: 0 to 1926


In [ ]:
VOCAB_SIZE = 1928 # 1927 moves and one padding token
MAX_SEQUENCE_LENGTH = 6 # for now 
PAD_TOKEN = 1927 # for not known moves

class chessdataset(Dataset):
    
    def __init__(self, games_data, max_seq_len = 6):
        self.max_seq_len = max_seq_len
        self.pad_token = PAD_TOKEN
        self.sequences = []
        self.targets = []
        
        # loop through each games 
        for games in games_data:
            for i in  range(len(games) -1):
                start_idx = max(0,i-5)
                sequence = game[start_idx : i+1]  
            
            while len(sequence) <6:
                sequence.insert(0,(0,1927,0))

            target = game[i+1][1] # the real seventh move 

            self.sequences.append(sequence)
            self.targets.append(target)



    
    def __len__ (self):
        return None
        
                 

    def __getitem__ (self, idx):
        return None



                     
                

IndentationError: expected an indented block after 'for' statement on line 13 (345133522.py, line 20)